# Project 1 — Seattle Airbnb Open Data

## 1 Introduction & Background

### 1.1 Business Context

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Curabitur mollis euismod turpis vel ultrices. Nullam sollicitudin urna sapien, sed gravida odio pulvinar non. Praesent nec massa nec mi cursus cursus ac non sapien. Fusce ligula ipsum, tristique gravida luctus nec, semper tempus dolor. 

### 1.2 Questions

Lorem ipsum dolor sit amet, consectetur adipiscing elit.

#### Question 1

Which areas are most expensive?

#### Question 2

Is there a correlation between number of photos and bookings?

Is there a correlation between price and positive reviews?

#### Question 3

Build model to predict monthly price of apartments. Which factors have the biggest influence?

## 2 Analysis

Lorem ipsum dolor sit amet, consectetur adipiscing elit.

### 2.1 Getting Started & Data Gathering

#### Import Packages

In [1]:
import os
import sys
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns  
from IPython import display 
%matplotlib inline
print('Imported packages')

Imported packages


#### Read Listings Data

In [2]:
df_listings = pd.read_csv('./Data/Seattle Airbnb Open Data/listings.csv') 
df_listings.head(2)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",...,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48


In [3]:
df_listings.shape

(3818, 92)

In [11]:
df_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 92 columns):
id                                  3818 non-null int64
listing_url                         3818 non-null object
scrape_id                           3818 non-null int64
last_scraped                        3818 non-null object
name                                3818 non-null object
summary                             3641 non-null object
space                               3249 non-null object
description                         3818 non-null object
experiences_offered                 3818 non-null object
neighborhood_overview               2786 non-null object
notes                               2212 non-null object
transit                             2884 non-null object
thumbnail_url                       3498 non-null object
medium_url                          3498 non-null object
picture_url                         3818 non-null object
xl_picture_url                      3498

#### Read Calendar Data

In [5]:
df_calendar = pd.read_csv('./Data/Seattle Airbnb Open Data/calendar.csv') 
df_calendar.head(2) 

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00


In [6]:
df_calendar.shape

(1393570, 4)

In [12]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1393570 entries, 0 to 1393569
Data columns (total 4 columns):
listing_id    1393570 non-null int64
date          1393570 non-null object
available     1393570 non-null object
price         934542 non-null object
dtypes: int64(1), object(3)
memory usage: 42.5+ MB


#### Read Reviews Data

In [8]:
df_reviews = pd.read_csv('./Data/Seattle Airbnb Open Data/reviews.csv') 
df_reviews.head(2) 

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...


In [9]:
df_reviews.shape

(84849, 6)

In [10]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84849 entries, 0 to 84848
Data columns (total 6 columns):
listing_id       84849 non-null int64
id               84849 non-null int64
date             84849 non-null object
reviewer_id      84849 non-null int64
reviewer_name    84849 non-null object
comments         84831 non-null object
dtypes: int64(3), object(3)
memory usage: 3.9+ MB


### 2.2 Data Wrangling

#### Check for null values

Listings Data: Percent of missing values in each column

In [39]:
percent_missing = df_listings.isnull().sum() * 100 / len(df_listings)
missing_value_listings = pd.DataFrame({'column_name': df_listings.columns,
                                 'percent_missing': percent_missing})

missing_value_listings.sort_values('percent_missing', inplace=True)

print(missing_value_listings)
missing_value_listings.tail(10)

                       column_name  percent_missing
id                              id         0.000000
price                        price         0.000000
amenities                amenities         0.000000
bed_type                  bed_type         0.000000
accommodates          accommodates         0.000000
...                            ...              ...
weekly_price          weekly_price        47.380828
security_deposit  security_deposit        51.126244
monthly_price        monthly_price        60.267156
square_feet            square_feet        97.459403
license                    license       100.000000

[92 rows x 2 columns]


,column_name,percent_missing
host_about,host_about,22.498690
transit,transit,24.463070
cleaning_fee,cleaning_fee,26.977475
neighborhood_overview,neighborhood_overview,27.029859
notes,notes,42.063908
weekly_price,weekly_price,47.380828
security_deposit,security_deposit,51.126244
monthly_price,monthly_price,60.267156
square_feet,square_feet,97.459403
license,license,100.000000


Drop columns **license** and **square feet** completely, as they have almost exclusively missing data.
The column **neighbourhood** can be dropped, as it contains null values, while the column **neighbourhood_cleansed** has the same information, just without missing data and in the correct format.

In [90]:
df_listings_clean = df_listings.drop(columns=['license', 'square_feet', 'neighbourhood'])

print("Original dataframe:")
print(df_listings.shape)
print("New dataframe:")
print(df_listings_clean.shape)

Original dataframe:
(3818, 92)
New dataframe:
(3818, 89)


Drop rows where **neighbourhood** or **monthly_price** are missing, as we are highly interested in those metrics, especially for building a supervised learning model.

In [91]:
df_listings_clean = df_listings_clean.dropna(subset=['monthly_price'], axis=0)
print("New dataframe:")
print(df_listings_clean.shape)

New dataframe:
(1517, 89)


Check again colums that have missing values

In [92]:
percent_missing = df_listings_clean.isnull().sum() * 100 / len(df_listings_clean)
missing_value_listings = pd.DataFrame({'column_name': df_listings_clean.columns,
                                 'percent_missing': percent_missing})

missing_value_listings.sort_values('percent_missing', inplace=True)
missing_value_listings.tail(15)

,column_name,percent_missing
first_review,first_review,10.283454
review_scores_rating,review_scores_rating,10.613052
review_scores_cleanliness,review_scores_cleanliness,10.678972
review_scores_communication,review_scores_communication,10.678972
review_scores_location,review_scores_location,10.810811
review_scores_value,review_scores_value,10.810811
review_scores_accuracy,review_scores_accuracy,10.876730
review_scores_checkin,review_scores_checkin,10.876730
host_acceptance_rate,host_acceptance_rate,13.183916
cleaning_fee,cleaning_fee,17.732367


In [98]:
df_listings_clean[['neighborhood_overview', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'zipcode']]

,neighborhood_overview,neighbourhood_cleansed,neighbourhood_group_cleansed,zipcode
1,"Queen Anne is a wonderful, truly functional vi...",West Queen Anne,Queen Anne,98119
3,NaN,West Queen Anne,Queen Anne,98119
8,Walking Score: 92 4 blocks from Kerry Park Fam...,West Queen Anne,Queen Anne,98109
9,"Queen Anne is a wonderful, truly functional vi...",West Queen Anne,Queen Anne,98119
11,I am located in the Upper Queen Anne neighborh...,West Queen Anne,Queen Anne,98119
...,...,...,...,...
3802,I loved living here before we had 3 kids. The ...,Fremont,Other neighborhoods,98103
3807,From the apartment it is an easy walk to the h...,Fremont,Other neighborhoods,98103
3809,This cottage apartment is located just a few b...,Fremont,Other neighborhoods,98103
3810,We love that Fremont is so centrally located t...,Fremont,Other neighborhoods,98103


In [123]:
print(df_listings_clean['neighbourhood_cleansed'].value_counts())
print('\n')
print('--------------')
print('\n')
print(df_listings_clean['neighbourhood_group_cleansed'].value_counts())

Broadway                  120
Wallingford                84
Belltown                   74
University District        68
Fremont                    58
                         ... 
Roxhill                     2
Industrial District         2
North Beach/Blue Ridge      2
Riverview                   2
Arbor Heights               1
Name: neighbourhood_cleansed, Length: 84, dtype: int64


--------------


Other neighborhoods    364
Downtown               181
Capitol Hill           173
Central Area           149
West Seattle            95
Queen Anne              93
Ballard                 78
Rainier Valley          73
University District     68
Northgate               45
Beacon Hill             43
Delridge                38
Cascade                 38
Magnolia                27
Lake City               26
Seward Park             21
Interbay                 5
Name: neighbourhood_group_cleansed, dtype: int64


### 2.3 Answering Q1

#### Question: Which area in the city is most expensive?

Having a look at columns related to area and neighborhood.

In [184]:
df_listings_clean[['neighborhood_overview', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'zipcode', 'monthly_price']]

,neighborhood_overview,neighbourhood_cleansed,neighbourhood_group_cleansed,zipcode,monthly_price
1,"Queen Anne is a wonderful, truly functional vi...",West Queen Anne,Queen Anne,98119,"$3,000.00"
3,NaN,West Queen Anne,Queen Anne,98119,"$2,300.00"
8,Walking Score: 92 4 blocks from Kerry Park Fam...,West Queen Anne,Queen Anne,98109,"$1,700.00"
9,"Queen Anne is a wonderful, truly functional vi...",West Queen Anne,Queen Anne,98119,"$3,000.00"
11,I am located in the Upper Queen Anne neighborh...,West Queen Anne,Queen Anne,98119,"$2,000.00"
...,...,...,...,...,...
3802,I loved living here before we had 3 kids. The ...,Fremont,Other neighborhoods,98103,"$5,400.00"
3807,From the apartment it is an easy walk to the h...,Fremont,Other neighborhoods,98103,"$1,800.00"
3809,This cottage apartment is located just a few b...,Fremont,Other neighborhoods,98103,"$2,200.00"
3810,We love that Fremont is so centrally located t...,Fremont,Other neighborhoods,98103,"$5,500.00"


In [193]:
df_listings_clean_monthly_price = df_listings_clean[['neighbourhood_cleansed', 'monthly_price']]
print(df_listings_clean_monthly_price)

     neighbourhood_cleansed monthly_price
1           West Queen Anne     $3,000.00
3           West Queen Anne     $2,300.00
8           West Queen Anne     $1,700.00
9           West Queen Anne     $3,000.00
11          West Queen Anne     $2,000.00
...                     ...           ...
3802                Fremont     $5,400.00
3807                Fremont     $1,800.00
3809                Fremont     $2,200.00
3810                Fremont     $5,500.00
3812                Fremont     $2,200.00

[1517 rows x 2 columns]


In [197]:
float(df_listings_clean_monthly_price['monthly_price'][1])

ValueError: could not convert string to float: '$3,000.00'

In [191]:
df_listings_clean_monthly_price.groupby(['neighbourhood_cleansed']).agg(['mean'])

ValueError: no results

In [192]:
df_listings_clean.groupby(['neighbourhood_cleansed']).agg(['mean'])

,id,scrape_id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,...,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,reviews_per_month
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,...,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
neighbourhood_cleansed,,,,,,,,,,,,,,,,,,,,,
Adams,4.283482e+06,20160104002432,1.486263e+07,1.782609,1.782609,47.671895,-122.384806,3.826087,1.413043,1.478261,...,41.565217,96.526316,9.789474,9.842105,10.000000,10.000000,10.000000,9.631579,1.130435,3.371053
Alki,6.297037e+06,20160104002432,1.954027e+07,2.055556,2.055556,47.575439,-122.406182,4.000000,1.277778,1.666667,...,14.388889,95.500000,9.642857,9.785714,9.857143,9.785714,9.928571,9.357143,1.222222,1.920000
Arbor Heights,5.252543e+06,20160104002432,2.349883e+07,1.000000,1.000000,47.512482,-122.387147,2.000000,1.500000,1.000000,...,6.000000,100.000000,9.000000,10.000000,10.000000,10.000000,8.000000,9.000000,1.000000,0.850000
Atlantic,5.338848e+06,20160104002432,1.904100e+07,1.444444,1.444444,47.595957,-122.305800,3.518519,1.388889,1.259259,...,23.000000,93.956522,9.695652,9.521739,9.739130,9.608696,8.869565,9.434783,1.370370,2.523333
Belltown,5.816851e+06,20160104002432,1.523485e+07,24.438356,24.438356,47.615169,-122.345621,3.500000,1.116438,0.958904,...,31.162162,93.164179,9.522388,9.567164,9.582090,9.671642,9.835821,9.179104,3.635135,2.817015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
West Woodland,4.862326e+06,20160104002432,1.324843e+07,1.409091,1.409091,47.668478,-122.366791,3.272727,1.272727,1.227273,...,40.272727,97.000000,10.000000,9.842105,9.947368,10.000000,9.789474,9.789474,1.045455,2.980526
Westlake,3.032020e+06,20160104002432,1.418827e+07,1.750000,1.750000,47.633404,-122.342487,3.500000,1.250000,1.250000,...,12.750000,92.500000,9.250000,9.250000,9.250000,9.500000,9.500000,9.000000,1.500000,2.400000


In [176]:
df_listings_clean.shape

(1517, 89)

In [174]:
df_listings_clean.groupby(['neighbourhood_cleansed']).agg(['mean', 'count']).columns

MultiIndex([(                            'id',  'mean'),
            (                            'id', 'count'),
            (                     'scrape_id',  'mean'),
            (                     'scrape_id', 'count'),
            (                       'host_id',  'mean'),
            (                       'host_id', 'count'),
            (           'host_listings_count',  'mean'),
            (           'host_listings_count', 'count'),
            (     'host_total_listings_count',  'mean'),
            (     'host_total_listings_count', 'count'),
            (                      'latitude',  'mean'),
            (                      'latitude', 'count'),
            (                     'longitude',  'mean'),
            (                     'longitude', 'count'),
            (                  'accommodates',  'mean'),
            (                  'accommodates', 'count'),
            (                     'bathrooms',  'mean'),
            (                  

In [156]:
df_neighbourhood_price = df_listings_clean.groupby(['neighbourhood_cleansed']).agg(['mean', 'count'])[[('review_scores_communication',  'mean'), ('review_scores_communication',  'count')]]

KeyError: "None of [MultiIndex([('monthly_price',  'mean'),\n            ('monthly_price', 'count')],\n           )] are in the [columns]"

In [157]:
print(df_neighbourhood_price)

                       review_scores_communication      
                                              mean count
neighbourhood_cleansed                                  
Adams                                    10.000000    19
Alki                                      9.785714    14
Arbor Heights                            10.000000     1
Atlantic                                  9.608696    23
Belltown                                  9.671642    67
...                                            ...   ...
West Woodland                            10.000000    19
Westlake                                  9.500000     4
Whittier Heights                         10.000000     9
Windermere                                9.833333     6
Yesler Terrace                           10.000000     8

[84 rows x 2 columns]
